In [2]:
%cd ~/code/projects/nn-play/

import keras 
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam, Adadelta, RMSprop, SGD
from keras.layers import Input, Dense, Convolution2D, Deconvolution2D, MaxPooling2D,\
    UpSampling2D, Merge, LSTM, Flatten, ZeroPadding2D, Reshape, BatchNormalization, Dropout
from keras.layers.wrappers import TimeDistributed

from structured_container import DataContainer

import tqdm
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

/home/ira/code/projects/nn-play


Using TensorFlow backend.


In [3]:
IM_WIDTH = 28
IM_HEIGHT = 28
IM_CHANNELS = 1

SERIES_SHIFT = 1
EP_LEN = 100 - SERIES_SHIFT
BATCH_SIZE = 32

V_SIZE = 128

empty_model = True

In [4]:
input_im = Input(shape=(IM_WIDTH, IM_HEIGHT, IM_CHANNELS))
h = Convolution2D(16, 5, 5, subsample=(2,2), activation='relu', border_mode='same')(input_im)
h = Convolution2D(8, 3, 3, subsample=(2,2), activation='relu', border_mode='same')(h)
h = Flatten()(h)
v = Dense(V_SIZE, activation='relu')(h)

enc = Model(input_im, v)
# enc.summary()

In [5]:
input_v = Input(shape=(V_SIZE,))

h = Dense(8*7*7, activation='relu')(input_v)
h = Reshape((7,7,8))(h)
h = UpSampling2D((2,2))(h)
h = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(h)
h = UpSampling2D((2,2))(h)
output_im = Convolution2D(1, 5, 5, activation='sigmoid', border_mode='same')(h)

# output_im

dec = Model(input_v, output_im)
# dec.summary()

In [6]:
input_im = Input(shape=(IM_WIDTH, IM_HEIGHT, IM_CHANNELS))
v = enc(input_im)
output_im = dec(v)

ae = Model(input_im, output_im)
# ae.compile(optimizer=Adam(lr=0.0002), loss='mse')
# ae.summary();

In [7]:
pred_ae = Sequential()
pred_ae.add(TimeDistributed(enc, input_shape=((EP_LEN-SERIES_SHIFT, IM_WIDTH, IM_HEIGHT, IM_CHANNELS))))
pred_ae.add(LSTM(V_SIZE, input_shape=(EP_LEN, V_SIZE), return_sequences=True))

pred_ae.add(TimeDistributed(dec, input_shape=((EP_LEN, V_SIZE))))

pred_ae.compile(optimizer=Adam(lr=0.001), loss='mse')
# pred_ae.summary();

In [398]:
pred_ae.compile(optimizer=Adam(lr=0.0002), loss='mse')

In [ ]:
images_train = DataContainer('data-balls/balls-train.pt', batch_size=32,
                          ep_len_read=EP_LEN)
images_valid = DataContainer('data-balls/balls-valid.pt', batch_size=32,
                          ep_len_read=EP_LEN)

In [ ]:
images_train.populate_images()

In [ ]:
x = images_train.get_batch_episodes()
i = 0

In [ ]:
plt.imshow(x[0,i,:,:,0], cmap='gray', vmin=0.0, vmax=1.0)
i += 1

In [ ]:
if empty_model:
    fpath = 'models/pred_ae_single_init.hdf5'
#     pred_ae.save_weights(fpath)
    pred_ae.load_weights(fpath)
    empty_model = False

In [38]:
def validate(p):
    x = images_valid.get_batch_episodes()
    x_noised = np.copy(x)
    for_removal = np.random.random(EP_LEN) < p
    for_removal[0:15] = False
    x_noised[:, for_removal] = 0
    test_perf = pred_ae.test_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])
    print('p', p)
    print('test perf', test_perf)

In [57]:
# train without noise
for i in tqdm.tqdm(range(2000)):
   x = images_train.get_batch_episodes()
   pred_ae.train_on_batch(x[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])

100%|██████████| 2000/2000 [05:59<00:00,  5.69it/s]


In [422]:
validate(1.0)

p 1.0
test perf 0.00435068


In [96]:
# train with noise
for p in np.linspace(0.72, 0.9, 10):
    for j in range(100):
        x = images_train.get_batch_episodes()
        x_noised = np.copy(x)
        for_removal = np.random.random(EP_LEN) < p
        for_removal[0:15] = False
        x_noised[:, for_removal] = 0
        pred_ae.train_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])

        
    validate(1.0)

p 1.0
test perf 0.0302881
p 1.0
test perf 0.0295654
p 1.0
test perf 0.0316305
p 1.0
test perf 0.028337
p 1.0
test perf 0.0321089
p 1.0
test perf 0.0268758
p 1.0
test perf 0.027803
p 1.0
test perf 0.0274918
p 1.0
test perf 0.0259274
p 1.0
test perf 0.0273803


In [97]:
p

0.90000000000000002

In [437]:
x = images_valid.get_batch_episodes()
x_noised = np.copy(x)
for_removal = np.random.random(EP_LEN) < 1.0
for_removal[0:4] = False
x_noised[:, for_removal] = 0
pred_ae.test_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])
y = pred_ae.predict(x_noised[:,0:-SERIES_SHIFT,...])
i = SERIES_SHIFT

In [438]:
# gif generation
import imageio
images = []
for i in range(EP_LEN-SERIES_SHIFT):
    im_true = x[0,i+SERIES_SHIFT,:,:,0]
    im_noised = x_noised[0,i+SERIES_SHIFT,:,:,0]
    im_gen = y[0,i,:,:,0]
    im_diff = np.abs(im_true - im_gen)
    glued = np.concatenate([im_true,
                            im_gen,
                            im_noised,
                            im_diff],
                           axis=1)
# plt.imshow(glued, cmap='gray')
    images.append(glued)
imageio.mimsave('movie.gif', images)

/home/ira/code/envs/flexi/lib/python3.5/site-packages/imageio/core/util.py:78: UserWarning: Lossy conversion from float64 to uint8, range [0, 1]
  dtype_str, out_type.__name__))


In [415]:
p = 1.0
for j in tqdm.tqdm(range(2000)):
    x = images_train.get_n_random_episodes(32)
    x_noised = np.copy(x)
    for_removal = np.random.random(EP_LEN) < p
    for_removal[0:3 + np.random.randint(0,20)] = False
    x_noised[:, for_removal] = 0
    pred_ae.train_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])


x = images_valid.get_n_random_episodes(32)
x_noised = np.copy(x)
for_removal = np.random.random(EP_LEN) < p
for_removal[0:15] = False
x_noised[:, for_removal] = 0
test_perf = pred_ae.test_on_batch(x_noised[:,0:-SERIES_SHIFT,...], x[:,SERIES_SHIFT:,...])
print('p', p)
print('test perf', test_perf)

 91%|█████████ | 1821/2000 [05:23<00:32,  5.54it/s]

KeyboardInterrupt: 

In [439]:
if not empty_model:
    fpath = 'models/pred_ae_single_trained_2.hdf5'
    pred_ae.save_weights(fpath)